# Create mangrove blue carbon asset

Based on [ee code](https://code.earthengine.google.com/b1e2d678541dc3e9ff46eaf85df175c9)

The export grid used has been [calculated](../download-grid-creation.ipynb) as 
the max allowed pixel size for the GEE export: 1e9 pixels

This code creates a raster imageCollection representing the total carbon
(in CO2e units) stock of global mangrove ecotypes per hectare (t CO2e / ha).  

Input data:  
* soc: mangrove soil organic carbon t OC / m2
* agb: above ground biomass t / m2

Conversions:
* soc: t OC / m2 -> t CO2e / m2  
* agb: t / m2 -> t CO2e / m2
* bgb: agb t / m2 -> bgb  t / m2 -> t CO2e / m2

Factors:
* Above to Below Ground Biomass: 0.49 (Simard et al. 2019)
* Biomass to Carbon: 0.451 (Simard et al. 2019)
* Organic Carbon to CO2e: 11/3 = 3.66 

In [12]:
import os
from pathlib import Path
import json
import ee
import geemap
import geopandas as gpd
from IPython.display import display, Markdown, JSON
import shapely.speedups


%run utils.ipynb

In [16]:
# Trigger the authentication flow.
#ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate(auth_mode='notebook')
# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


### Data Assets

In [18]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets/raw'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets/raw', f'{BASE_DIR} is not the correct directory'

# variables
version = 'v2_1'
base_gcs_url = 'https://storage.googleapis.com/mangrove_atlas/ee_export_tiffs'
dataset = 'mangrove_blue_carbon'
# SET EXPORT REGION
region = ee.Geometry.Polygon([-180, 33, 0,33,180, 33,180,-40,0,-40,-180,-40], None, False)
# SET piramiding policy
piramiding_policies = {'total_co2e': 'mean',
                     'agb_co2e': 'mean',
                     'bgb_co2e': 'mean',
                     'soc_co2e': 'mean'}

In [19]:
## Assets selection
#agb: above-ground biomass density
agb = ee.Image(ee.ImageCollection(
    "projects/global-mangrove-watch/mangrove-properties/mangrove_aboveground_biomass_1996-2016").first())
# oc : soil organic carbon
soc = ee.Image("projects/global-mangrove-watch/mangrove-properties/mangroves_SOC30m_0_100cm")

### Create the Blue carbon asset

In [20]:
# SET CONSTANTS
# factor to convert above-ground biomass to below-ground biomass (Simard et al. 2019)
agb_to_bgb = ee.Number(0.49)
# factor to convert above-ground biomass to organic carbon (Simard et al. 2019)
biomass_to_OC = ee.Number(0.451)
# factor to convert organic carbon to CO2 equivalent
OC_to_CO2e = ee.Number(11).divide(3)

maxPixels = 1e16

nms = soc.projection().nominalScale().getInfo()

In [21]:
def organic_c_to_co2(oc: ee.Image) -> ee.Image:
    '''
    Convert organic carbon to CO2 equivalent
    '''
    return oc.multiply(OC_to_CO2e)

def biomass_to_co2(biomass: ee.Image) -> ee.Image:
    '''
    Convert above-ground biomass to CO2 equivalent
    '''
    return organic_c_to_co2(biomass.multiply(biomass_to_OC))

def above_to_below(agb: ee.Image) -> ee.Image:
    '''
    Convert above-ground biomass to below-ground biomass
    '''
    return agb.multiply(agb_to_bgb)

def calculate_blue_carbon(soc: ee.Image, abg: ee.Image):
    """
    Calculate the blue carbon from soil organic carbon and the above ground biomass.
    """
    bgb = above_to_below(abg)
    soc_co2 = (organic_c_to_co2(soc.updateMask(soc.gt(0)))).rename('soc_co2e')
    abg_co2 = (biomass_to_co2(abg.updateMask(abg.gt(0)))).rename('agb_co2e')
    bgb_co2 = (biomass_to_co2(bgb)).rename('bgb_co2e')
    
    # Return the image with the blue carbon.
    return ee.Image(soc_co2.add(abg_co2).add(bgb_co2)).rename('total_co2e').addBands(soc_co2).addBands(abg_co2).addBands(bgb_co2)


In [22]:
def check_asset(asset_id: str)-> bool:
    """
    Check if an asset exists in GEE.
    """
    try:
        ee.data.getAsset(asset_id)
        return True
    except:
        # TODO: this can fail becouse the asset does not exist or the user does 
        # not have rights to read the asset
        return False

In [52]:
def create_blue_carbon_asset(dataset: str, data_version: str):
    """
    Create an asset with the gain/loss of the mangrove extent
    """
    ee_asset_path = f'projects/global-mangrove-watch/mangrove-properties/{dataset}-{data_version}'
    
    description = f"""
    # Mangrove blue carbon

    ## Methodology

    This dataset shows the amount and density of carbon stored in mangrove biomass and soil. Total values represent the sum of aboveground carbon, and the soil organic carbon values, attributed to mangrove forests. 
    Total values are expressed in Megatonnes of carbon dioxide equivalents (Mt CO2e), while mapped carbon density values are depicted as metric tonnes of CO2 equivalents per hectare (t CO2e / ha). Above-ground estimates of mangrove carbon were obtained from Simard et al. 2019. The data were derived from remotely-sensed canopy height measurements and region-specific allometric models validated using in-situ measurements in field plots across three continents. This was converted to mean AGB carbon using the stoichiometric factor of 0.451 (Simard et al. 2019). Soil Organic Carbon (SOC) estimates of the top meter of mangrove soils based on a methodology developed by Sanderman et al. (2018) were overlaid with 2016 mangrove extent maps from Bunting et al. 2018 to produce a global map of soil organic carbon at a 30 m spatial resolution. Aboveground and soil carbon values originally expressed in metric tonnes (megagrams) of carbon per hectare were converted to total carbon using the Bunting et al. (2018) mangrove extent for each country and converted to MtCO2e using a conversion factor of 3.67 (Howard et al. 2014).
    """

    collection_properties = ImageCollectionProperties(
        name = f"Mangrove blue carbon ({data_version})",
        version = data_version,
        creator = "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
        description = description,
        identifier = "",
        keywords = "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
        citation = "Bunting, P., Rosenqvist, A., Lucas, R. M., Rebelo, L. M., Hilarides, L., Thomas, N., … Finlayson, C. M. (2018). The global mangrove watch - A new 2010 global baseline of mangrove extent. Remote Sensing, 10(10).  \
        Giri, C., Ochieng, E., Tieszen, L. L., Zhu, Z., Singh, A., Loveland, T., … Duke, N. (2011). Status and distribution of mangrove forests of the world using earth observation satellite data. Global Ecology and Biogeography, 20(1), 154–159. https://doi.org/10.1111/j.1466-8238.2010.00584.x  \
        Sanderman J, Hengl T, Fiske G et al. (2018) A global map of mangrove forest soil carbon at 30 m spatial resolution. Environmental Research Letters 13: 055002. doi: 10.1088/1748-9326/aabe1c  \
        Simard, M., T. Fatoyinbo, C. Smetanka, V.H. Rivera-monroy, E. Castaneda-mova, N. Thomas, and T. Van der stocken. (2019) Global Mangrove Distribution, Aboveground Biomass, and Canopy Height. ORNL DAAC, Oak Ridge, Tennessee, USA.",
        license = "https://creativecommons.org/licenses/by/4.0/",
        url = "",
        language = "en", 
        altName = f"Global Mangrove Watch, {data_version}",
        distribution = "",
        variableMeasured = "Blue carbon of mangrove forests",
        units = "1",
        spatialCoverage = "Global tropics",
        temporalCoverage = '2016',
        dataLineage = "Raster data supplied by Aberystwyth University (Dr. Dave Bunting) as tilesets per year, \
            each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
    )

    return ee.data.createAsset({"type":"ImageCollection"}, ee_asset_path, collection_properties.dict(exclude_none=True))

def create_image(asset: ee.ImageCollection, asset_name: str, region: ee.geometry, pyramiding_policy: dict, year: int, nms: int):
        
        name = f"{asset_name}/{year}"
        
        image_properties = ImageProperties(
                    band_nodata_values = 0,
                    band_pyramiding_policies = 'MEAN',
                    band_names = ', '.join(pyramiding_policy.keys()),
                    year = year,
                )
        print(f"export_{asset_name}_{str(year)}")
        params = {
                'image': asset.filterMetadata('year', 'equals', year).first().set(image_properties.dict(exclude_none=True)),
                'description': f"export_{asset_name.split('/')[-1]}_{year}",
                'assetId': name,
                'pyramidingPolicy': pyramiding_policy,
                'scale': nms,
                'crs': 'EPSG:4326',
                'region': region,
                'maxPixels': 1e13
                }
        # if not check_asset(asset_name):
        #     ee.data.createAsset({"type":"Image"}, asset_name, image_properties.dict(exclude_none=True))
        # else:
        #     ee.data.updateAsset(asset_name, image_properties.dict(exclude_none=True))
        
        
        return ee.batch.Export.image.toAsset(**params)

def exportAssetTasks(asset: ee.ImageCollection, asset_name: str, geometry: ee.geometry, pyramiding_policy: dict,  years: list = [2016], view = True, export = False):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    geometry : ee.geometry
        The geometry to export.
    pyramiding_policy : dict, optional
        The pyramiding policy. The default is {'total_co2e': 'mean'}.
    years : list, optional
        The years to export. The default is [2016].
        
    
    Returns
    -------
    List of tasks
    """
    taskList = []
    collection_name = f'projects/global-mangrove-watch/mangrove-properties/{asset_name}-{version}'
    
    # TODO: improve this loop with a map function
    if export:
        if not check_asset(collection_name):
            create_blue_carbon_asset(dataset, version)
            
        nms = asset.first().select('total_co2e').projection().nominalScale().getInfo()
        for year in years:
            task = create_image(asset, collection_name, geometry, pyramiding_policy, year, nms)
            task.start()
            taskList.append(task)
    
    if view:
        display(Markdown(f"### Map {asset_name}"))
        # Create a geemap  object.
        Map = geemap.Map(center=(-7.998, 39.4767), zoom=9, basemap='HYBRID')
        vis_params = {
            'bands': ['total_co2e'],
            'min': 0,
            'max': 3500,
            'palette': ['#5c4a3d',
                        '#933a06',
                        '#b84e17',
                        '#e68518',
                        '#eeb66b'],
        }
        Map.addLayer(asset.first(), vis_params, asset_name, True, 1)
        display(Map)
    
    return taskList

In [36]:
blue_carbon = calculate_blue_carbon(soc, agb).set({"year":2016})
JSON(blue_carbon.getInfo())

<IPython.core.display.JSON object>

In [53]:
tasks = exportAssetTasks(ee.ImageCollection.fromImages([blue_carbon]), dataset, region, piramiding_policies, years = [2016], view = True, export = True)

export_projects/global-mangrove-watch/mangrove-properties/mangrove_blue_carbon-v2_1_2016


### Map mangrove_blue_carbon

Map(center=[-7.998, 39.4767], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [54]:
# Check task status
!earthengine task list | head -n 20

G3YQCNS2CIKSAD6TOC3IQCK7  Export.image  export_mangrove_blue_carbon-v2_1_2016  RUNNING    ---
7O6JRV5XJXXJVM4Y65O3QPH5  Export.image  export_mangrove_blue_carbon-v2_1_2016  FAILED     Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.


In [56]:
tasks[0].status()

{'state': 'RUNNING',
 'description': 'export_mangrove_blue_carbon-v2_1_2016',
 'creation_timestamp_ms': 1686758464981,
 'update_timestamp_ms': 1686759009617,
 'start_timestamp_ms': 1686758468075,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'G3YQCNS2CIKSAD6TOC3IQCK7',
 'name': 'projects/earthengine-legacy/operations/G3YQCNS2CIKSAD6TOC3IQCK7'}

## Export the data to GCS

In [13]:
# Grid for download export
grid_filename = Path(f'{BASE_DIR}/grid_test.shp')
export_tiles = gpd.read_file(grid_filename)
export_tiles['url'] = export_tiles.apply(lambda row: f'{base_gcs_url}/{dataset}/2016/{row.id}.tif', axis=1)
fc = geemap.geopandas_to_ee(export_tiles)

In [34]:
def exportDataTasks(asset: ee.ImageCollection, asset_name: str, gcbucket: str, geometry_collection: gpd.GeoDataFrame):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    List of tasks
    """
    fc = geemap.geopandas_to_ee(geometry_collection)

    nms = asset.first().select('total_co2e').projection().nominalScale().getInfo()
    years = asset.date().getInfo()['year']
    taskList = []
    
    # TODO: improve this loop
    
    for geometry_id in geometry_collection:
        for year in years:
            task = ee.batch.Export.image.toCloudStorage(
                image = ee.Image(asset.filterMetadata('year', 'equals', year)).clip(
                    fc.filterMetadata('id', 'equals', geometry_id).geometry()),
                bucket =gcbucket,
                fileNamePrefix = f'ee_export_tiffs/{asset_name}/{year}/{geometry_id}',
                description = f'{asset_name}_{year}_{geometry_id}',
                scale = nms,
                region = fc.filterMetadata('id', 'equals', geometry_id).geometry(),
                fileFormat = 'GeoTIFF', 
                maxPixels = 1e13,
                formatOptions = {'cloudOptimized': True})
            
            taskList.append(task)
    
    return taskList

In [35]:
def batchExecute(taskList: list, batch_size: int = 20):
    """
    Execute the tasks in the list in baches of 20 as is the max allowed by GEE. 
    each task takes about 10 min to execute.
    """
    n_tasks = len(taskList)
    n_batches = Math.ceil(n_tasks / batch_size)
    
    for i in range(n_batches):
        for task in taskList[i*batch_size:(i+1)*batch_size]:
            task.start()

    return taskList

In [48]:
# This is how we can dynamically create a url for downloading the data. there are some constrains on the nº of pixels that can be exported.
print(blue_carbon.getDownloadUrl({
    'name': 'blue_carbon',
    'scale': nms,
    'crs': 'EPSG:4326',
    'region': '[[39.184112548828125,-6.525458832948283],[39.494476318359375,-6.525458832948283],[39.494476318359375,-6.255236941610093],[39.184112548828125,-6.255236941610093],[39.184112548828125,-6.525458832948283]]'
}))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ab79e79dd89bfc40b03baa84e74fe293-69efccdd85237a841e0f974e7e0233e8:getPixels


In [164]:
# This is how you can export the data as a table if you need to create statistics

# data = blue_carbon.reduceRegions(
#     collection=fc,
#     reducer=ee.Reducer.sum()
# )
# task =  ee.batch.Export.table.toCloudStorage( 
#    collection = data,
#  bucket = 'mangrove-carbon',
#  fileNamePrefix = 'ee_export_tiffs/blue_carbon'
# )
# task.start()

In [209]:
#dimensions = '1000000000000x1000000000000',
#skipEmptyTiles = True,
# a max of 20 at a time
task = ee.batch.Export.image.toCloudStorage(
           image = blue_carbon.clip(fc.filterMetadata('id', 'equals', 4).geometry()),
           bucket ='mangrove_atlas',
           fileNamePrefix = f'ee_export_tiffs/test_total_organic_carbon-3',
           description = 'test_total_organic_carbon-2',
           scale = nms,
           region = fc.filterMetadata('id', 'equals', 4).geometry(),
           fileFormat = 'GeoTIFF', 
           maxPixels = 1e13,
           formatOptions = {'cloudOptimized': True})
task.start()
task.id

'R3DQ6J4GYDBPF35CIOO5C6FB'

In [210]:
task.status()

{'state': 'READY',
 'description': 'test_total_organic_carbon-2',
 'creation_timestamp_ms': 1658840921125,
 'update_timestamp_ms': 1658840921125,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'R3DQ6J4GYDBPF35CIOO5C6FB',
 'name': 'projects/earthengine-legacy/operations/R3DQ6J4GYDBPF35CIOO5C6FB'}